In [6]:
# Components OBS script

#Importar Librerias 
import pandas as pd
from datetime import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains


In [7]:
import pandas as pd


file_path = r'C:\Users\chaconri\Desktop\Components\Componentes Test.xlsx'
components_data = pd.read_excel(file_path)
components_data



,Components
0,C7977-60000
1,5184-2414
2,Q2079AN
3,Q2078-60000
4,5184-2416
...,...
4994,P26775-B21
4995,P26909-001
4996,P36188-B21
4997,P22903-001


In [8]:
# Convert DataFrame data to a list of lists
Components = components_data.values.tolist()
print(Components)

[['C7977-60000'], ['5184-2414'], ['Q2079AN'], ['Q2078-60000'], ['5184-2416'], ['Q2079AL'], ['C7975-60000'], ['5184-2488'], ['C7976-60000'], ['5184-2497'], ['866987-001'], ['866988-001'], ['867573-001'], ['867556-001'], ['866612-001'], ['866613-001'], ['866646-001'], ['867553-001'], ['869898-001'], ['872953-021'], ['872954-021'], ['307480-021'], ['703828-032'], ['868357-021'], ['106128-025'], ['855652-005'], ['675251-001'], ['868783-001'], ['868788-001'], ['330499-003'], ['291292-003'], ['290959-003'], ['99-07097-02'], ['404828-001'], ['404447-001'], ['392455-001'], ['404446-001'], ['404447-002'], ['392455-002'], ['106128-020'], ['106584-059'], ['106128-001'], ['106584-056'], ['869923-001'], ['438690-003'], ['876635-001'], ['876636-001'], ['876610-001'], ['876359-001'], ['876360-001'], ['876436-001'], ['291292-004'], ['871219-001'], ['871220-001'], ['871220-003'], ['871220-004'], ['871220-005'], ['871220-006'], ['871220-007'], ['871220-008'], ['871220-009'], ['398927-001'], ['398928-001

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import threading

def process_component(component):
    try:
        PMG_button = driver.find_element(By.ID, "chkPMG")
        PMG_button.click()
        
        prod_hierarchy = driver.find_element(By.XPATH, '//*[@id="lstReportType"]/option[3]')
        prod_hierarchy.click()

        SKU_input_text = driver.find_element(By.ID, "txtPartNumber")
        
        if component == 'N/A':
            obs_date = 'N/A'
        else:
            SKU_input_text.click()
            SKU_input_text.clear()
            SKU_input_text.send_keys(component)
            SKU_input_text.send_keys(Keys.ENTER)
            
            obs_date_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "dgPartLookup_lblPHObsDate_0")))
            obs_date = obs_date_element.text
        
        return [component, obs_date]
    except:
        return [component, 'N/A']

def process_components_range(start, end):
    processed_data = []
    for component in Components[start:end]:
        processed_data.append(process_component(component))
    
    return processed_data

# Lista de componentes obtenida del DataFrame
Components = components_data.values.tolist()

# Declaramos el objeto webdriver y se añade el driver (.exe) de Chrome en la ruta adecuada.
driver = webdriver.Chrome(executable_path='tu_ruta_al_executable_de_ChromeDriver')

# Indicamos la página a abrir.
driver.get("https://mypimproweb.its.hpecorp.net/PartLookup/PartLookup.aspx")

# Número de threads a utilizar (puede ajustarse según el rendimiento y capacidad de la máquina)
num_threads = 4
batch_size = len(Components) // num_threads

threads = []
results = []

# Crear y lanzar los threads
for i in range(num_threads):
    start = i * batch_size
    end = (i + 1) * batch_size if i != num_threads - 1 else len(Components)
    
    thread = threading.Thread(target=lambda s=start, e=end: results.extend(process_components_range(s, e)))
    threads.append(thread)
    thread.start()

# Esperar a que todos los threads terminen
for thread in threads:
    thread.join()

driver.quit()

# Crear el DataFrame con los datos recopilados.
OBS_df = pd.DataFrame(results, columns=["Component", "OBS Date"])

# Imprimir el DataFrame.
print(OBS_df)


C:\Users\chaconri\AppData\Local\Temp\ipykernel_8288\859033800.py:45: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='tu_ruta_al_executable_de_ChromeDriver')


         Component     OBS Date
0     [P45139-013]          N/A
1     [P45139-014]          N/A
2     [P45543-B21]  31-Dec-2030
3     [P45138-008]          N/A
4     [P45138-009]          N/A
...            ...          ...
4994  [P45139-008]          N/A
4995  [P45139-009]          N/A
4996  [P45139-010]          N/A
4997  [P45139-011]          N/A
4998  [P45139-012]          N/A

[4999 rows x 2 columns]


In [12]:
# Remove brackets and surrounding whitespace from the first column of OBS_df
OBS_df["Component"] = OBS_df["Component"].astype(str).str.replace(r'^[\[\]\s]+|[\[\]\s]+$', '', regex=True)
OBS_df

,Component,OBS Date
0,'P45139-013',N/A
1,'P45139-014',N/A
2,'P45543-B21',31-Dec-2030
3,'P45138-008',N/A
4,'P45138-009',N/A
...,...,...
4994,'P45139-008',N/A
4995,'P45139-009',N/A
4996,'P45139-010',N/A
4997,'P45139-011',N/A


In [13]:
# Remove single quotes from the first column of OBS_df
OBS_df["Component"] = OBS_df["Component"].str.replace("'", "")
OBS_df

,Component,OBS Date
0,P45139-013,N/A
1,P45139-014,N/A
2,P45543-B21,31-Dec-2030
3,P45138-008,N/A
4,P45138-009,N/A
...,...,...
4994,P45139-008,N/A
4995,P45139-009,N/A
4996,P45139-010,N/A
4997,P45139-011,N/A


In [14]:

output_file = r'C:\Users\chaconri\Desktop\Components\ComponentsOBS.xlsx'
# Export the DataFrame to Excel
OBS_df.to_excel(output_file, index=False)
